In [ ]:
# !pip install efficientnet_pytorch

# PyTorch Lightning

In this notebook I have tried using pytorch-lightning, I was amazed to see the features it provides ,i spent some time going through the docs and implement the basics of it, i am yet to go through and implement the advanced features it provides which can help in finding the optimal hyperparameters.

I have used EfficientNet-b5 trained model, saved the efficientnet_pytorch package wheel (.whl) file as this competition involved no internet. 

### Accuracy: 88% on public test images 

In [ ]:
import sys

In [ ]:
efficientnet_path='../input/efficientnetlib/efficientnet_pytorch-0.7.0-py3-none-any.whl'

sys.path.append(efficientnet_path)


# Importing Libraries


In [ ]:
import os
import sys
import pandas
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score

import cv2
import albumentations as albu
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnext50_32x4d
from torch.utils.data import Dataset,DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
from pytorch_lightning.metrics.functional import accuracy
from efficientnet_pytorch import EfficientNet


# Constants

In [ ]:
TRAIN_IMAGES_DIR='../input/cassava-leaf-disease-classification/train_images'
TEST_IMAGES_DIR='../input/cassava-leaf-disease-classification/test_images'
TRAIN_CSV='../input/cassava-leaf-disease-classification/train.csv'
PRETRAINED_PATH='../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth'
BATCH_SIZE=8
IMG_SIZE=512
CLASSES=5

# LightningModule

* Use LightningModule as base class and create your own training class
* Output model in forward method 
* Use configure_optimizers for adding any optimization and scheduler
* Use training_step for computing and returning loss (everything else like calculating gradient, updating params and device setting is handled by Lightning )
* Similarly validation_step for inference
* Add logging using .log


In [ ]:
class CassavaLite(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.efficient_net = EfficientNet.from_name('efficientnet-b5')
        self.efficient_net.load_state_dict(torch.load(PRETRAINED_PATH))
#         self.efficient_net=EfficientNet.from_pretrained('efficientnet-b3',num_classes=CLASSES)
        in_features=self.efficient_net._fc.in_features
        self.efficient_net._fc=nn.Linear(in_features,CLASSES)
    
    def forward(self,x):
        out=self.efficient_net(x)
        return out
    
    def configure_optimizers(self):
        optimizer=torch.optim.AdamW(self.parameters(),lr=1e-4,weight_decay=0.0001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
        return [optimizer],[scheduler]
    
    def training_step(self,batch,batch_idx):
        x,y=batch["x"],batch["y"]
        y_hat=self(x)
        loss=F.cross_entropy(y_hat,y)
        acc=accuracy(y_hat,y)
        self.log("train_acc",acc,on_step=False,on_epoch=True,prog_bar=True,logger=True)
        self.log("train_loss",loss,on_step=False,on_epoch=True,prog_bar=True,logger=True)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x,y=batch["x"],batch["y"]
        y_hat=self(x)
        loss=F.cross_entropy(y_hat,y)
        acc=accuracy(y_hat,y)
        self.log("val_acc",acc,prog_bar=True,logger=True),
        self.log("val_loss",loss,prog_bar=True,logger=True)
        

# Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        self.transforms=transforms
        
    def __getitem__(self,index):
        im_path=os.path.join(self.imfolder,self.df.iloc[index]['image_id'])
        x=cv2.imread(im_path,cv2.IMREAD_COLOR)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        
        if(self.transforms):
            x=self.transforms(image=x)['image']
        
        if(self.train):
            y=self.df.iloc[index]['label']
            return {
                    "x":x,
                    "y":y,
                }    
        else:
            return {"x":x}
        
    def __len__(self):
        return len(self.df)

# Lightning Data Module

LightningDataModule can be used to prepare and dispatch data, which can then be passed in fit method of Trainer.

In [ ]:
class CassavaDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train_transform = albu.Compose([
                        albu.RandomResizedCrop(IMG_SIZE,IMG_SIZE, p=1.0),
                        albu.Transpose(p=0.5),
                        albu.HorizontalFlip(p=0.5),
                        albu.VerticalFlip(p=0.5),
                        albu.ShiftScaleRotate(p=0.5),
                        albu.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
                        albu.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                        albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                        albu.CoarseDropout(p=0.5),
                        albu.Cutout(p=0.5),
                        ToTensorV2(p=1.0),
                        ], p=1.)

        self.valid_transform = albu.Compose([
                         albu.CenterCrop(IMG_SIZE,IMG_SIZE, p=1.),
                         albu.Resize(IMG_SIZE, IMG_SIZE),
                         albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                         ToTensorV2(p=1.0),
                         ], p=1.)
        
        
    def prepare_data(self):
        # prepare_data is called only once on 1- GPU in a distributed computing
        df=pd.read_csv(TRAIN_CSV)
        df["kfold"]=-1
        df=df.sample(frac=1).reset_index(drop=True)
        stratify=StratifiedKFold(n_splits=5)
        for i,(t_idx,v_idx) in enumerate(stratify.split(X=df.image_id.values,y=df.label.values)):
            df.loc[v_idx,"kfold"]=i
            df.to_csv("train_folds.csv",index=False)
    
    def setup(self,stage=None):
        dfx=pd.read_csv("train_folds.csv")
        train=dfx.loc[dfx["kfold"]!=1]
        val=dfx.loc[dfx["kfold"]==1]
        
        self.train_dataset=CassavaDataset(
                            train,
                            TRAIN_IMAGES_DIR,
                            train=True,
                            transforms=self.train_transform)
        
        self.valid_dataset=CassavaDataset(
                            val,
                            TRAIN_IMAGES_DIR,
                            train=True,
                            transforms=self.valid_transform)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.valid_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4)
                         

In [ ]:
model_checkpoint=ModelCheckpoint(monitor="val_loss",
                                 verbose=True,
                                 filename="{epoch}_{val_loss:.4f}")
early_stopping = EarlyStopping('val_loss',patience=4)

In [ ]:
# early_stop_callback=True
# model=resnext50_32x4d(pretrained=True)#Add Pretrained=True to use pretrained with internet enabled
dm=CassavaDataModule()
cassava_model=CassavaLite()
trainer=pl.Trainer(gpus=-1,max_epochs=12,callbacks=[model_checkpoint,early_stopping])
trainer.fit(cassava_model,dm)

#manually you can save best checkpoints - 
trainer.save_checkpoint("cassava_efficient_net.ckpt")

In [ ]:
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")

test_transform = albu.Compose([
                    albu.RandomResizedCrop(IMG_SIZE,IMG_SIZE),
                    albu.Transpose(p=0.5),
                    albu.HorizontalFlip(p=0.5),
                    albu.VerticalFlip(p=0.5),
                    albu.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
                    albu.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                    albu.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                    ToTensorV2(p=1.0),
                    ], p=1.)


In [ ]:
test_dataset = CassavaDataset(test_df,
                            TEST_IMAGES_DIR,
                            train=False,
                            transforms=test_transform)

test_loader = DataLoader(test_dataset,
                        batch_size=16)

In [ ]:

# model=resnext50_32x4d()#Add Pretrained=True to use pretrained with internet enabled
best_checkpoints = trainer.checkpoint_callback.best_model_path
pretrained_model = CassavaLite().load_from_checkpoint(checkpoint_path = best_checkpoints)
pretrained_model = pretrained_model.to("cuda")
pretrained_model.eval()
pretrained_model.freeze()

fin_out = []
for data in test_loader:
    y_hat = pretrained_model(data["x"].to("cuda"))
    y_hat = torch.argmax(y_hat,dim=1)
    fin_out.extend(y_hat.cpu().detach().numpy().tolist())
test_df["label"] = fin_out
test_df[["image_id","label"]].to_csv("submission.csv",index=False)
test_df.head()